# **Predictive Default Risk Assessor V.01**

# TODO

* Base model 
* Comparison
* Specialised
* For small entities - Examples?
* Backtest - All sectors 
* Understanding the model across all sectors/industries
* Any markets - consumer goods, industries
* UI last step after backtesting

In [173]:
model_inputs = {
    "profitability": {
        "class_weight": 0.05,
        "weights": [1.0], 
        "metrics": {
            "oper_margin": {
                "lower_is_better": False,
                "thresholds": [
                    (40, float("inf")),
                    (35, 39),
                    (30, 34),
                    (25, 29),
                    (20, 24),
                    (15, 19),
                    (10, 14),
                    (5, 9),
                    (float("-inf"), 0)
                ],
            }
        },
    },
    "leverage_coverage": {
        "class_weight": 0.71,
        "weights": [0.2, 0.32, 0.48],
        "metrics": {
            "tot_debt_to_tot_eqy": {
                "lower_is_better": True,
                "thresholds": [
                     (float("-inf"), 2.0),
                     (2.0, 16.0),
                     (16.0, 24.0),
                     (24.0, 33.0),
                     (33.0, 43.0),
                     (43.0, 54.0),
                     (54.0, 68.0),
                     (68.0, 94.0),
                     (94.0, float("inf")),
                ],
            },
            "tot_debt_to_ebitda": {
                "lower_is_better": True,
                "thresholds": [
                    (float("-inf"), 0.09),
                    (0.09, 0.49),
                    (0.49, 0.9),
                    (0.9, 1.36),
                    (1.36, 1.68),
                    (1.68, 2.26),
                    (2.26, 3.27),
                    (3.27, 4.4),
                    (4.4, float("inf")),
                ],
            },
            "ebitda_to_tot_int_exp": {
                "lower_is_better": False,
                "thresholds": [
                    (25, float("inf")),
                    (20, 25),
                    (15, 20),
                    (10, 15),
                    (5, 10),
                    (3, 5),
                    (1, 3),
                    (0, 1),
                    (float("-inf"), 0),
                ],
            },
        },
    },
    "efficiency": {
        "class_weight": 0.25,
        "weights": [0.6, 0.5],
        "metrics": {
            "return_on_asset": {
                "lower_is_better": False,
                "thresholds": [
                    (0.15, float("inf")),
                    (0.10, 0.15),
                    (0.08, 0.10),
                    (0.06, 0.08),
                    (0.04, 0.06),
                    (0.02, 0.04),
                    (0.00, 0.02),
                    (-0.02, 0.00),
                    (float("-inf"), -0.02)
                ],
            },
            "asset_turnover": {
                "lower_is_better": False,
                "thresholds": [
                    (4.0, float("inf")),
                    (3.0, 4.0),
                    (2.0, 3.0),
                    (1.5, 2.0),
                    (1.0, 1.5),
                    (0.75, 1.0),
                    (0.5, 0.75),
                    (0.25, 0.5),
                    (float("-inf"), 0.25)
                ],
            },
        },
    },
}

In [2]:
import pandas as pd
import numpy as np

In [15]:
class CreditRatingCalculator:
    def __init__(self, metrics):
        self.metrics = metrics
        
    def _calculate_metric_score(self, metric, thresholds, inverse):
        for score, (lower, upper) in enumerate(thresholds, start=1):
            if (inverse and metric <= upper) or (not inverse and metric >= lower):
                return score
        return len(thresholds) // 2 # else return the middle score

    def _calculate_category_score(self, category_metrics, ratios):
        total_weighted_score = 0

        for metric, weight in zip(
            category_metrics["metrics"].items(), category_metrics["weights"]
        ):
            metric_name, metric_data = metric
            value = ratios[metric_name]
            score = self._calculate_metric_score(
                value, metric_data["thresholds"], metric_data["lower_is_better"]
            )
            total_weighted_score += score * weight

        return total_weighted_score

    def _calculate_scores(self, ratios):
        scores = {}
        for category, category_data in self.metrics.items():
            category_score = self._calculate_category_score(category_data, ratios)
            scores[category] = category_score
        return scores

    def _calculate_weighted_score(self, scores):
        weights = {
            category: category_data["class_weight"]
            for category, category_data in self.metrics.items()
        }
        return sum(scores[category] * weight for category, weight in weights.items())

    def _determine_credit_rating(self, weighted_score):
        credit_ratings = [
            (2.5, "Aaa"),
            (3.5, "Aa"),
            (4.5, "A"),
            (5.5, "Baa"),
            (6.5, "Ba"),
            (7.5, "B"),
            (8.5, "Caa"),
            (9.5, "Ca"),
            (float("inf"), "C"),
        ]

        for threshold, rating in credit_ratings:
            if weighted_score < threshold:
                return rating

    def calculate_credit_rating(self, ratios):
        self.scores = self._calculate_scores(ratios)
        self.credit_score = self._calculate_weighted_score(self.scores)
        self.credit_rating = self._determine_credit_rating(self.credit_score)

In [55]:
features = pd.read_excel("dataset/features.xlsx", index_col=0)
targets = pd.read_excel("dataset/target.xlsx", index_col=0)
features.columns = features.columns.str.lower()

In [56]:
targets

,credit_rating,rating,numeric_rating
AGL SJ Equity,Baa2,Baa,5.5
ANG SJ Equity,Baa3,Baa,5.5
ANH SJ Equity,A3,A,4.5
BAW SJ Equity,Ba2,Ba,6.5
BHG SJ Equity,A1,A,4.5
...,...,...,...
UGPA3 BS Equity,Ba1,Ba,6.5
USIM5 BS Equity,Ba2,Ba,6.5
VALE3 BS Equity,Baa3,Baa,5.5
VAMO3 BS Equity,BB-,Ba,6.5


In [33]:
features.index

Index(['AGL SJ Equity', 'ANG SJ Equity', 'ANH SJ Equity', 'BAW SJ Equity',
       'BHG SJ Equity', 'BTI SJ Equity', 'BVT SJ Equity', 'CFR SJ Equity',
       'DSY SJ Equity', 'FFB SJ Equity', 'FSR SJ Equity', 'GFI SJ Equity',
       'GLN SJ Equity', 'GRT SJ Equity', 'HMN SJ Equity', 'MNP SJ Equity',
       'MSP SJ Equity', 'MTN SJ Equity', 'NRP SJ Equity', 'PPH SJ Equity',
       'PRX SJ Equity', 'RDF SJ Equity', 'S32 SJ Equity', 'SAP SJ Equity',
       'SNT SJ Equity', 'SOL SJ Equity', 'SSW SJ Equity', 'TKG SJ Equity',
       'ABEV3 BS Equity', 'AZUL4 BS Equity', 'B3SA3 BS Equity',
       'BBAS3 BS Equity', 'BBDC3 BS Equity', 'BBDC4 BS Equity',
       'BEEF3 BS Equity', 'BPAC11 BS Equity', 'BRFS3 BS Equity',
       'BRKM5 BS Equity', 'CIEL3 BS Equity', 'CMIG4 BS Equity',
       'CSAN3 BS Equity', 'CSNA3 BS Equity', 'ELET3 BS Equity',
       'ELET6 BS Equity', 'EMBR3 BS Equity', 'ENGI11 BS Equity',
       'GGBR4 BS Equity', 'HYPE3 BS Equity', 'JBSS3 BS Equity',
       'KLBN11 BS Equity'

In [37]:
company = "RRRP3 BS Equity"

In [39]:
features.loc[company]

asset_turnover                        0.4001
bs_lt_borrow                       8329.6270
bs_st_borrow                        977.8530
bs_total_liabilities              14554.7680
bs_tot_asset                      20119.8580
cfo_to_tot_debt                       0.1084
ebitda                             1885.9700
ebitda_to_interest_expn               2.8302
ebitda_to_revenue                    33.5583
ebitda_to_tot_int_exp                 2.8302
fcf_to_total_debt                     0.0221
is_oper_inc                        1265.8700
net_debt_to_ebitda                    2.7013
oper_margin                          22.5244
retained_cash_flow_to_net_debt        0.1980
return_on_asset                       2.8852
return_on_cap                         8.9074
return_on_inv_capital                 9.2942
sales_growth                        226.2841
sales_rev_turn                     5619.9890
short_and_long_term_debt           9307.4800
total_equity                       5565.0900
tot_debt_t

In [40]:
model_metrics = [
    "oper_margin", 
    "tot_debt_to_tot_eqy", 
    "tot_debt_to_ebitda", 
    "ebitda_to_tot_int_exp", 
    "return_on_asset",
    "asset_turnover",
]

ratios = features.loc[company][model_metrics].to_dict()

In [41]:
ratios

{'oper_margin': 22.5244,
 'tot_debt_to_tot_eqy': 167.2476,
 'tot_debt_to_ebitda': 4.9351,
 'ebitda_to_tot_int_exp': 2.8302,
 'return_on_asset': 2.8852,
 'asset_turnover': 0.4001}

In [174]:
model = CreditRatingCalculator(model_inputs)
model.calculate_credit_rating(ratios)
print(f"Model Inputs:")
display(ratios)
print("")
print(f"Class Scoring: {model.scores}")
print(f"Credit Score: {model.credit_score}")
print(f"Credit Rating: {model.credit_rating}")

Model Inputs:


{'oper_margin': 4.372,
 'tot_debt_to_tot_eqy': 98.646,
 'tot_debt_to_ebitda': 2.0211,
 'ebitda_to_tot_int_exp': 5.5277,
 'return_on_asset': 11.2683,
 'asset_turnover': 3.8526}


Class Scoring: {'profitability': 9.0, 'leverage_coverage': 5.751826676095, 'efficiency': 1.72076757}
Credit Score: 7.278178546837023
Credit Rating: B


In [177]:
study = {}
ra = {}
for company in features.index:
    ratios = features.loc[company][model_metrics].to_dict()
    model = CreditRatingCalculator(model_inputs)
    model.calculate_credit_rating(ratios)
    ra[company] = ratios
    study[company] = {"class_scoreing": model.scores, "credit_score": model.credit_score, "credit_rating": model.credit_rating}
    
study = pd.DataFrame(study).T

In [178]:
study

,class_scoreing,credit_score,credit_rating
AGL SJ Equity,"{'profitability': 7.0, 'leverage_coverage': 6....",6.996391,B
ANG SJ Equity,"{'profitability': 9.0, 'leverage_coverage': 5....",5.44147,Baa
ANH SJ Equity,"{'profitability': 5.0, 'leverage_coverage': 7....",7.768269,Caa
BAW SJ Equity,"{'profitability': 8.0, 'leverage_coverage': 5....",6.596693,B
BHG SJ Equity,"{'profitability': 1.0, 'leverage_coverage': 2....",1.80309,Aaa
...,...,...,...
UGPA3 BS Equity,"{'profitability': 9.0, 'leverage_coverage': 6....",8.51446,Ca
USIM5 BS Equity,"{'profitability': 9.0, 'leverage_coverage': 7....",8.619837,Ca
VALE3 BS Equity,"{'profitability': 3.0, 'leverage_coverage': 3....",2.344731,Aaa
VAMO3 BS Equity,"{'profitability': 3.0, 'leverage_coverage': 7....",7.914067,Caa


In [179]:
ms = ['AGL SJ Equity', 'ANG SJ Equity', 'ANH SJ Equity', 'BAW SJ Equity',
'BHG SJ Equity', 'BVT SJ Equity', 'GFI SJ Equity', 'MNP SJ Equity',
'SAP SJ Equity', 'SOL SJ Equity', 'TKG SJ Equity', 'ABEV3 BS Equity',
'AZUL4 BS Equity', 'B3SA3 BS Equity', 'BEEF3 BS Equity',
'BPAC11 BS Equity', 'BRFS3 BS Equity', 'BRKM5 BS Equity',
'CIEL3 BS Equity', 'CMIG4 BS Equity', 'CSAN3 BS Equity',
'CSNA3 BS Equity', 'ELET3 BS Equity', 'ELET6 BS Equity',
'EMBR3 BS Equity', 'ENGI11 BS Equity', 'GGBR4 BS Equity',
'HYPE3 BS Equity', 'JBSS3 BS Equity', 'KLBN11 BS Equity',
'MOVI3 BS Equity', 'MRFG3 BS Equity', 'NTCO3 BS Equity',
'PETR3 BS Equity', 'PETR4 BS Equity', 'PRIO3 BS Equity',
'RAIL3 BS Equity', 'RAIZ4 BS Equity', 'RDOR3 BS Equity',
'RENT3 BS Equity', 'RRRP3 BS Equity', 'SBSP3 BS Equity',
'SMTO3 BS Equity', 'SUZB3 BS Equity', 'UGPA3 BS Equity',
'USIM5 BS Equity', 'VALE3 BS Equity', 'VAMO3 BS Equity',
'VBBR3 BS Equity']

In [180]:
import pandas as pd

In [181]:
ta = pd.read_excel('t.xlsx', index_col=0)

In [182]:
ta

,numeric_rating
AGL SJ Equity,5.5
ANG SJ Equity,5.5
ANH SJ Equity,4.5
BAW SJ Equity,6.5
BHG SJ Equity,4.5
BVT SJ Equity,6.5
GFI SJ Equity,5.5
MNP SJ Equity,5.5
SAP SJ Equity,6.5
SOL SJ Equity,6.5


In [183]:
a = study.loc[ms].join(ta)

In [184]:
a['credit_score'] = a['credit_score'].astype(float).round(1)

In [185]:
y_true = a['numeric_rating']
yhat = a['credit_score']

In [189]:
y_true

AGL SJ Equity       5.5
ANG SJ Equity       5.5
ANH SJ Equity       4.5
BAW SJ Equity       6.5
BHG SJ Equity       4.5
BVT SJ Equity       6.5
GFI SJ Equity       5.5
MNP SJ Equity       5.5
SAP SJ Equity       6.5
SOL SJ Equity       6.5
TKG SJ Equity       6.5
ABEV3 BS Equity     5.5
AZUL4 BS Equity     8.5
B3SA3 BS Equity     6.5
BEEF3 BS Equity     6.5
BPAC11 BS Equity    6.5
BRFS3 BS Equity     6.5
BRKM5 BS Equity     6.5
CIEL3 BS Equity     6.5
CMIG4 BS Equity     6.5
CSAN3 BS Equity     6.5
CSNA3 BS Equity     6.5
ELET3 BS Equity     6.5
ELET6 BS Equity     6.5
EMBR3 BS Equity     6.5
ENGI11 BS Equity    6.5
GGBR4 BS Equity     5.5
HYPE3 BS Equity     6.5
JBSS3 BS Equity     5.5
KLBN11 BS Equity    6.5
MOVI3 BS Equity     6.5
MRFG3 BS Equity     6.5
NTCO3 BS Equity     6.5
PETR3 BS Equity     6.5
PETR4 BS Equity     6.5
PRIO3 BS Equity     6.5
RAIL3 BS Equity     6.5
RAIZ4 BS Equity     5.5
RDOR3 BS Equity     6.5
RENT3 BS Equity     6.5
RRRP3 BS Equity     7.5
SBSP3 BS Equity 

In [186]:
from sklearn.metrics import mean_squared_error

In [190]:
mean_squared_error(y_true, yhat)

4.37265306122449

In [137]:
X = pd.DataFrame(ra).T.loc[y_true.index].values

In [146]:
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

In [192]:
import numpy as np
from scipy.optimize import minimize

def optimize_weights(X, y):
    
    def error_function(weights):
        predicted_y = np.dot(X, weights)
        error = np.mean((predicted_y - y) ** 2)  # Mean Squared Error
        return -mean_absolute_percentage_error(y, predicted_y)
    
    bounds = [(None, 1)] * X.shape[1]
    constraint = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
    result = minimize(error_function, np.ones(X.shape[1]), constraints=constraint)

    # Extract the optimal weights
    optimal_weights = result.x
    print(result)
    return optimal_weights

# Obtain optimal weights
optimal_weights = optimize_weights(X, y_true)
print("Optimal weights:", optimal_weights)


 message: Rank-deficient equality constraint subproblem HFTI
 success: False
  status: 7
     fun: -2.6359436323636383e+18
       x: [ 6.363e+15  7.779e+16 -1.273e+16 -1.273e+16 -3.143e+16
           -2.727e+16]
     nit: 68
     jac: [-3.096e+00 -3.421e+01 -5.853e-01 -9.975e-01 -6.788e-01
           -1.169e-01]
    nfev: 535
    njev: 68
Optimal weights: [ 6.36277822e+15  7.77927332e+16 -1.27255686e+16 -1.27255576e+16
 -3.14346704e+16 -2.72697153e+16]


In [193]:
optimal_weights

array([ 6.36277822e+15,  7.77927332e+16, -1.27255686e+16, -1.27255576e+16,
       -3.14346704e+16, -2.72697153e+16])

In [171]:
a = optimal_weights[4:]
print(a.sum())
a /= a.sum()
print(a)

-0.3148423265868361
[0.27923243 0.72076757]


In [172]:
a

array([0.27923243, 0.72076757])

In [123]:
import numpy as np

def compute_error(X, y, w):
    y_pred = np.dot(X, w)
    error = np.mean((y_pred - y) ** 2)
    return error

def gradient_descent(X, y, learning_rate=0.01, epochs=1000):
    num_features = X.shape[1]
    w = np.random.randn(num_features) # initialize weights randomly
    for _ in range(epochs):
        y_pred = np.dot(X, w)
        error = compute_error(X, y, w)
        gradient = 2 * np.dot(X.T, (y_pred - y)) / len(y)
        w -= learning_rate * gradient
    return w

# Example usage:
# X is the feature matrix, y is the target vector
# Make sure X has shape (num_samples, num_features) and y has shape (num_samples,)
# Call gradient_descent function to get optimized weights

In [124]:
optimized_weights = gradient_descent(X, y_true)


C:\Users\Thabang Ndhlovu\AppData\Local\Temp\ipykernel_11132\538052255.py:15: RuntimeWarning: invalid value encountered in subtract
  w -= learning_rate * gradient


In [125]:
optimized_weights

array([nan, nan, nan, nan, nan, nan])